<a href="https://colab.research.google.com/github/DipankarJDutta/Biomarker-discovery-for-FASD-GDM-with-B-T-cells/blob/Optimizing-dropout-rate-and-weight-constraint/B-T-median_optimizing%20dropout%20rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Loading essentials
import numpy
from numpy import loadtxt
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

Using TensorFlow backend.


In [0]:
# Fixing random seed for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [0]:
#Creating model required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
    #create model
    model = Sequential()
    model.add(Dense(27, input_dim=29, kernel_initializer='glorot_normal', activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(39, kernel_initializer='glorot_normal', activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
#load dataset
dataset = loadtxt ('B-T-median.csv', delimiter = ',')

In [0]:
#split dataset into input and output variables
x = dataset [:, 0:29]
y = dataset [:, 29]

In [0]:
#Create model for gridsearch
model = KerasClassifier (build_fn=create_model, epochs=240, batch_size=5, verbose=0)

In [0]:
#Define Grid Search parameters for optimal Batch size for input T-cell dataset with 47 samples
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
#Jobs are run in parallel with n_jobs=-1; 10-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=-1, cv=10)
grid_result = grid.fit(x, y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [0]:
#Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.676667 using {'dropout_rate': 0.1, 'weight_constraint': 3}
0.566667 (0.165328) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.590000 (0.170000) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.520000 (0.150702) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.586667 (0.208273) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.550000 (0.162788) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.550000 (0.185742) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.590000 (0.110604) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.676667 (0.106510) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.550000 (0.193936) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.530000 (0.167631) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.520000 (0.150702) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.533333 (0.158465) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.643333 (0.148361) with: {'dropout_rate': 0.2, 'weight_constraint': 